In [1]:
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit
import numpy as np
import pandas as pd

df_train = pd.read_csv('df_train.csv', index_col='row_id')
df_test = pd.read_csv('df_test.csv', index_col='row_id')
target_train = pd.read_csv('target_train.csv', index_col='row_id').squeeze()
target_test = pd.read_csv('target_test.csv', index_col='row_id').squeeze()

In [2]:
def smape_loss(y_true, y_pred):
    """
    Custom SMAPE loss function for LightGBM
    """
    return np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred))) * 100

def xgb_smape_eval(y_pred, y_true):
    y_true = y_true.get_label()
    return 'SMAPE', smape_loss(y_true, y_pred)

def smape_cv(model, df_train, target_train):
    tscv = TimeSeriesSplit(n_splits=5)
    smape_list = []
    for _, (train_index, test_index) in enumerate(tscv.split(df_train), start=1):
        # Create training and validation datasets
        X_train = df_train.iloc[train_index]
        y_train = target_train.iloc[train_index]
        X_valid = df_train.iloc[test_index]
        y_valid = target_train.iloc[test_index]

        model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric=xgb_smape_eval, verbose=False)

        # Make predictions
        y_pred = np.exp(model.predict(X_valid)) * X_valid['microbusiness_density']
        y_valid = np.exp(y_valid) * X_valid['microbusiness_density']

        # Calculate SMAPE score
        smape = smape_loss(y_valid, y_pred)
        smape_list.append(smape)

    print(f'Mean SMAPE: {np.mean(smape_list):.4f}')
    return np.mean(smape_list)

def xgb_objective(trial, df_train, target_train):
    """
    Objective function for XGBoost optimization without rolling window cross-validation.
    """
    params = {
        'n_estimators': 200,
        'verbosity': 0,
        'objective': 'reg:squarederror',
        'random_state': 42,
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.1, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'alpha': trial.suggest_float('alpha', 1e-2, 10.0),
        'lambda': trial.suggest_float('lambda', 1e-2, 10.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 5, 250),
    }

    model = xgb.XGBRegressor(**params)
    score = smape_cv(model, df_train, target_train)
    return score

In [3]:
import optuna

# Optimize the XGBoost model
xgb_study = optuna.create_study(direction='minimize')
xgb_study.optimize(lambda trial: xgb_objective(trial, df_train, target_train), n_trials=20)

# Print the best trial for XGBoost
print(f"XGBoost best trial: {xgb_study.best_trial.value} with parameters {xgb_study.best_trial.params}")


[I 2023-04-16 04:07:39,409] A new study created in memory with name: no-name-6b691d0c-60f4-46c8-b810-d51344c6fb12
/opt/homebrew/Caskroom/miniforge/base/envs/base_env/lib/python3.8/site-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
[I 2023-04-16 04:08:08,833] Trial 0 finished with value: 1.8864803834509267 and parameters: {'colsample_bytree': 0.12134592038068481, 'colsample_bynode': 0.7465569808584384, 'max_depth': 10, 'learning_rate': 0.02497674143276924, 'alpha': 7.309729803932332, 'lambda': 2.0324445135383695, 'min_child_weight': 186}. Best is trial 0 with value: 1.8864803834509267.


Mean SMAPE: 1.8865


[I 2023-04-16 04:08:44,441] Trial 1 finished with value: 1.812176511859881 and parameters: {'colsample_bytree': 0.3369702902509209, 'colsample_bynode': 0.8741513757711705, 'max_depth': 3, 'learning_rate': 0.07108795537272454, 'alpha': 8.69679809574608, 'lambda': 3.738281801767208, 'min_child_weight': 43}. Best is trial 1 with value: 1.812176511859881.


Mean SMAPE: 1.8122


[I 2023-04-16 04:09:10,393] Trial 2 finished with value: 2.0957377560088113 and parameters: {'colsample_bytree': 0.4318908676707047, 'colsample_bynode': 0.6868024870813386, 'max_depth': 3, 'learning_rate': 0.02043114113600379, 'alpha': 8.604899792232104, 'lambda': 5.58533976523905, 'min_child_weight': 197}. Best is trial 1 with value: 1.812176511859881.


...

Mean SMAPE: 1.8077


[I 2023-04-16 04:21:13,334] Trial 18 finished with value: 1.7958464330764052 and parameters: {'colsample_bytree': 0.10685820513121513, 'colsample_bynode': 0.757182951942658, 'max_depth': 9, 'learning_rate': 0.06363704354933969, 'alpha': 6.522802336114437, 'lambda': 8.738845417207408, 'min_child_weight': 95}. Best is trial 18 with value: 1.7958464330764052.


Mean SMAPE: 1.7958


[I 2023-04-16 04:21:50,884] Trial 19 finished with value: 1.7951311298755097 and parameters: {'colsample_bytree': 0.10880224509618677, 'colsample_bynode': 0.7127090048295895, 'max_depth': 9, 'learning_rate': 0.06040546947751284, 'alpha': 6.330624917943333, 'lambda': 6.337656972390408, 'min_child_weight': 93}. Best is trial 19 with value: 1.7951311298755097.


Mean SMAPE: 1.7951
XGBoost best trial: 1.7951311298755097 with parameters {'colsample_bytree': 0.10880224509618677, 'colsample_bynode': 0.7127090048295895, 'max_depth': 9, 'learning_rate': 0.06040546947751284, 'alpha': 6.330624917943333, 'lambda': 6.337656972390408, 'min_child_weight': 93}


In [8]:
train_df = pd.read_csv('train.csv')
params = xgb_study.best_params
xgb_model = xgb.XGBRegressor(**params)

In [9]:
xgb_model.fit(df_train, target_train)
y_pred = np.exp(xgb_model.predict(df_test)) * df_test.microbusiness_density

In [10]:
my_cfips = df_train['cfips'].unique()
train = train_df[train_df.cfips.isin(my_cfips)]
pred = y_pred.values
y_true = train[train.row_id.str.contains('2022-07-01')].microbusiness_density.values
print(smape_loss(y_true, pred))

2.169519725462987


In [ ]:
# clip_err = True

# if clip_err:
#     df_record = y_pred.merge(y_base, on = 'row_id', how='inner')
#     df_record = df_record.merge(y_true, on = 'row_id', how='inner')
#     def smape_(y_true, y_pred):
#         return 100  * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))
    
#     base_err = df_record.apply(lambda x: smape_(x.y_true, x.y_base), axis=1)
#     pred_err = df_record.apply(lambda x: smape_(x.y_true, x.microbusiness_density), axis=1)
#     blacklist = base_err[(base_err + 1e-3) < pred_err].index
#     y_pred.iloc[blacklist] = y_base.iloc[blacklist]
#     avg_base_err = base_err.mean()
#     avg_pred_err = pred_err.mean()
#     print(avg_base_err, avg_pred_err)
#     df_sub = pd.read_csv('sample_submission.csv')
#     df_sub = df_sub.drop(columns='microbusiness_density')
#     df_sub['cfips'] = df_sub.row_id.str.split('_', expand=True)[0]
#     y_pred['cfips'] = y_pred.row_id.str.split('_', expand=True)[0]
#     merged_df = df_sub.merge(y_pred, on='cfips', how='left', suffixes=('', '_drop'))
#     merged_df = merged_df.loc[:,~merged_df.columns.str.endswith('_drop')]
#     # merged_df = merged_df.drop(columns='cfips')
# else:
#     df_sub = pd.read_csv('sample_submission.csv')
#     df_sub = df_sub.drop(columns='microbusiness_density')
#     df_sub['cfips'] = df_sub.row_id.str.split('_', expand=True)[0]
#     y_pred['cfips'] = y_pred.row_id.str.split('_', expand=True)[0]
#     merged_df = df_sub.merge(y_pred, on='cfips', how='left', suffixes=('', '_drop'))
#     merged_df = merged_df.loc[:,~merged_df.columns.str.endswith('_drop')]
#     # merged_df = merged_df.drop(columns='cfips')